Pandas là một package được sử dụng rộng rãi để xử lí và cấu trúc dữ liệu trong Python. Có rất nhiều bài hướng dẫn về nó nhưng mình vẫn muốn giới thiệu với các bạn một số mẹo hay để làm việc với Pandas.
## read_csv  
Mọi người chắc đều biết đến câu lệnh này. Nhưng nếu dữ liệu mà bạn muốn đọc lớn thì sao? Thử thêm tham số `nrows=5` để chỉ đọc một phần nhỏ của bảng trước khi đọc hết nguyên bảng có thể giúp chúng ta tránh được lỗi sai khi sử dụng nhầm dấu phân cách (chưa chắc lúc nào file csv cũng phân cách bằng dấu phẩy).  
Hoặc đối với linux chúng ta có thể kiểm tra 5 dòng đầu tiên với bất kì file text nào bằng câu lệnh `head -n 5 data.txt`.  
Sau đó các bạn có thể xuất ra tên của tất cả các cột bằng cách sử dụng `df.columns.tolist()` và sau đó thêm tham số `usecols=['c1', 'c2',...]` để sử dụng các cột mà bạn cần. Hơn thế nữa, nếu bạn biết kiểu dữ liệu của một số cột thì bạn có thể thêm tham số `dtype={'c1': str, 'c2': int, ...}` có thể giúp dữ liệu được load nhanh hơn. Một lợi ích nữa của tham số này đó là nếu bạn có một cột chứa cả chuỗi và số thì nên để kiểu dữ liệu của cột đó là string, bạn sẽ không gặp lỗi trong lúc merge các bảng sử dụng cột này là khóa.  
## select_dtypes
Nếu bạn tiền xử lí dữ liệu bằng Python, thì câu lệnh này có thể giúp bạn tiết kiệm thời gian. Sau khi đọc dữ liệu một bảng, kiểu dữ liệu mặc định của từng cột có thể là `bool`, `int64`, `float64`, `object`, `category`, `timedelta64` hoặc `datetime64`. Bạn có thể dùng câu lệnh `df.dtypes.value_counts` để xem tất cả các kiểu dữ liệu có thể có trong bảng sau đó dúng `df.select_dtypes(include=['float64', 'int64'])` để chọn một sub-dataframe chỉ chứa các dữ liệu là số.
## copy
Bạn nên biết tới câu lệnh này vì đây là một câu lệnh quan trọng. Nếu bạn làm theo đoạn lệnh dưới đây:
```python
import pandas as pd
df1 = pd.DataFrame({'a': [0,0,0], 'b': [1,1,1]})
df2 = df1
df2['a'] = df2['a'] + 1
df1.head()
```
Bạn sẽ thấy rằng `df1` cũng bị thay đổi bởi vì câu lệnh `df2 = df1` không tạo ra bản sao của `df1` và gán nó và `df2` mà lại tạo ra một con trỏ trỏ tới `df1`. Nên nếu có bất kì sự thay đổi nào ở `df2` thì `df1` cũng bị thay đổi. Để tạo ra một bản sao của `df1` thì chúng ta có thế làm bằng cách:
```
df2 = df1.copy()
```
hoặc là
```python
from copy import deepcopy
df2 = deepcopy(df1)
```
## map
Đây là một lệnh hay để biến đổi dữ liệu dạng đơn giản. Đầu tiên bạn tạo một dictionary với các keys là giá trị cũ và values là các giá trị mới.
```python
level_map = {1: 'high', 2: 'medium', 3: 'low'}
df['c_level'] = df['c'].map(level_map)
```
Một số ví dụ có thể áp dụng cách biến đổi này là: True, False thành 1, 0; định nghĩa các mức độ,...
## apply hoặc not apply
Nếu chúng ta muốn tạo một cột mới với đầu vào là một số cột khác thì hàm `apply` khá hữu dụng.
```python
import pandas as pd

def rule(x, y):
    if x == 'high' and y > 10:
        return 1
    else:
        return 0

df = pd.DataFrame({'c1': ['high', 'high', 'low', 'low'], 'c2': [0, 23, 17, 4]})
df['new'] = df.apply(lambda x: rule(x['c1'], x['c2']), axis = 1)
df.head()
```
Ở đoạn code trên, chúng ta tạo một hàm với hai tham số, và sử dụng hàm apply để tạo ra một cột mới với dữ liệu từ hai cột `c1` và `c2`.  
Nhưng có một vấn đề đối với `apply` đó là đôi lúc nó rất chậm. Ví dụ như chúng ta cần tính giá trị lớn nhất của 2 cột `c1` và `c2` thì chúng ta có thể làm như này:
```python
df['maximum'] = df.apply(lambda x: max(x['c1'], x['c2']), axis=1)
```
nhưng mà chúng ta sẽ thấy rằng nó chậm hơn câu lệnh này:
```python
df['maximum'] = df[['c1', 'c2']].max(axis=1)
```
Lưu ý: Đừng sử dụng apply nếu như có thể làm công việc đó với một hàm built-in khác (thường thì chúng sẽ nhanh hơn).
## value counts
Câu lệnh này được sử dụng để kiểm tra số lần lặp lại của các giá trị. Ví dụ, nếu muốn kiểm tra tất cả các giá trị có thể có và số lần lặp lại của mỗi giá trị trong cột `c`:
`df['c'].value_counts()`
Một số tham số hữu dụng của câu lệnh này đó là:  
`normalize = True` : Nếu như muốn kiểm tra tần số thay vì đếm số lượng.  
`dropna = False`: Nếu bạn muốn các giá trị bị thiếu cũng được hiển thị trong bảng thống kê.  
`df['c'].value_counts().reset_index()`: Nếu muốn chuyển từ bảng thống kê thành pandas dataframe để có thể sử dụng riêng.  
`f['c'].value_counts().reset_index().sort_values(by='index')`: Hiển thị bảng thống kê được sắp xếp theo các giá trị riêng biệt trong cột `c` thay vì đếm số lượng.
## Số lượng của các giá trị bị thiếu
Khi xây dựng các models, chúng ta sẽ muốn loại bỏ các dòng có quá nhiều giá trị bị thế hoặc tất cả giá trị đều bị thiếu. Chúng ta có thể sử dụng `isnull()` hoặc `sum()` để đếm số lượng các giá trị bị thiếu trong một số cột bằng cách:  
```python
import pandas as pd
import numpy as np
df = pd.DataFrame({ 'id': [1,2,3], 'c1':[0,0,np.nan], 'c2': [np.nan,1,1]})
df = df[['id', 'c1', 'c2']]
df['num_nulls'] = df[['c1', 'c2']].isnull().sum(axis=1)
df.head()
```
## Chọn các dòng với danh sách các IDs
Trong SQL chúng ta có thể làm việc này bằng cách sử dụng `SELECT * FROM … WHERE ID in (‘A001’, ‘C022’, …)` để lấy các dòng với danh sách các IDs. Nếu muốn làm điều tương tự với pandas, chúng ta làm như sau:
```python
df_filter = df['ID'].isin(['A001','C022',...])
df[df_filter]
```
## Tạo các nhóm theo phần trăm
Chúng ta có các cột chứa dữ liệu số và muốn phân loại các giá trị trong cột thành các nhóm, ví dụ như top 5% vào nhóm 1, 5-20% vào nhóm 2, 20-50% vào nhóm 3, từ 50% về sau vào nhóm 4. Dĩ nhiên chúng ta có thể làm việc này với `pandas.cut` nhưng đây sẽ là một lựa chọn khác:
```python
import numpy as np
cut_points = [np.percentile(df['c'], i) for i in [50, 80, 95]]
df['group'] = 1
for i in range(3):
    df['group'] = df['group'] + (df['c'] < cut_points[i])
# or <= cut_points[i]
```
Cách này sẽ chạy nhanh hơn (vì không cần sử dụng hàm `apply`).
## to_csv
Đây cũng là một câu lệnh mà mọi người sẽ sử dụng.  
Mẹo đầu tiên với `to-csv`:  
`print(df[:5].to_csv())`  
Chúng ta có thể sử dụng câu lệnh này để in ra 5 dòng đầu tiên của file csv mà chúng ta chuẩn bị lưu.  
Một mẹo khác đó là xử lí các giá trị số nguyên và các giá trị bị thiếu trộn lẫn với nhau. Nếu một cột chứa cả hai giá trị đó thì dữ liệu sẽ vẫn là float thay vì là int. Khi xuất bảng này, chúng ta có thể thêm `float_format='%.0f'` để làm tròn tất cả các giá trị floats thành integers. Dùng mẹo này nếu chỉ muốn outputs của tất cả cột là integer.